# Preparation of parsed function, docstring pairs from standard libraries

In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
in_path = "D:\PROJECT\data\CodeSearchNet\\aug_python_bleu_filter"
out_path = "D:\PROJECT\data\CodeSearchNet\\aug_python_bleu_clean"
filenames = ["train", "test", "valid"]

In [3]:
dataset_frame = {file : pd.read_json(os.path.join(in_path, file + ".jsonl"), lines=True) for file in filenames}

for k, v in tqdm(dataset_frame.items()):
    frame = v[['code', 'docstring']]
    frame.to_json(os.path.join(out_path, k + ".jsonl"), orient='records', lines=True)

100%|██████████| 3/3 [00:05<00:00,  1.71s/it]


# OLD

In [2]:
py_path = 'D:\PROJECT\data\out.json'
java_path = 'D:\PROJECT\scraper_tmp\out_java.json'

In [3]:
py_pairs = pd.DataFrame()
java_pairs = pd.DataFrame()

In [4]:
py_df = pd.read_json(py_path)
java_df = pd.read_json(java_path)

In [5]:
def extract(functions : pd.Series, type : str) -> pd.Series:
    return functions.map(lambda x : "\n".join([x.text() for x in parse.extract_comments_from_str(x, type) if len(x.text()) > 0]))

In [7]:
py_pairs['code'] = py_df.code
java_pairs['code'] = java_df.identifier

In [8]:
py_pairs['code'] = py_pairs['code'].map(lambda x : x.split(".")[-1] if "." in x else x)
java_pairs['code'] = java_pairs['code'].map(lambda x : x.split(".")[-1] if "." in x else x)

In [10]:
py_pairs['docstring'] = py_df.docstring
java_pairs['docstring'] = extract(java_df.function, 'text/x-java')

## Cleaning

In [11]:
def stopPhrase(x):
    remove = ["supplementary characters (LANG1300)", "not found", "NOI18N", "\n\n"]
    for phrase in remove:
        x = x.replace(phrase, "")
    return x if x != "\n" else ""

In [12]:
py_pairs['docstring'] = py_pairs['docstring'].apply(lambda x : stopPhrase(x))
java_pairs['docstring'] = java_pairs['docstring'].apply(lambda x : stopPhrase(x))

In [13]:
py_clean = py_pairs[py_pairs.docstring != ""].reset_index(drop=True)
java_clean = java_pairs[java_pairs.docstring != ""].reset_index(drop=True)

In [14]:
py_clean

,code,docstring
0,is_inplace,Whether we expect the setting to be in-place o...
1,val,NA values that should generally be valid_na fo...
2,test_clip_with_na_args,Should process np.nan argument as None
3,test_interpolate_timedelta_index,"Tests for non numerical index types - object,..."
4,get_freq,Find the appropriate frequency string to descr...
...,...,...
60376,on_key_press,Callback for key presses.
60377,on_mouse_move,Callback for mouse movements.
60378,test_clip_with_na_args,Should process np.nan argument as None
60379,test_interpolate_timedelta_index,"Tests for non numerical index types - object,..."


In [15]:
import os
augment_path = "D:\PROJECT\Augments"
java = "java.jsonl"
py = "python.jsonl"

java_clean.dropna().to_json(os.path.join(augment_path, java), orient="records", lines=True)
py_clean.dropna().to_json(os.path.join(augment_path, py), orient="records", lines=True)